## Imports

In [24]:
import pandas as pd 
import requests
import seaborn as sns
from datetime import datetime

from helper import clean_data
from helper import RMSE
from helper import split_data

from statsmodels.tsa.arima_model import ARIMA

import warnings
warnings.filterwarnings('ignore')

%pylab inline
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data loading & cleaning

In [25]:
weekly_pct, vix_close = clean_data()

In [26]:
train_vix, train_prime, validation, validation_prime, test = split_data(weekly_pct)

## SARIMAX

In [27]:
import statsmodels.api as sm

In [28]:
validation_prime_df = pd.DataFrame(validation_prime)
sar = sm.tsa.statespace.SARIMAX(train_vix, exog=train_prime, 
        order=(1,0,0), seasonal_order=(0,0,0,12), trend='c').fit()
pred = sar.forecast(52*2,exog=validation_prime_df)
RMSE(validation,pred)

0.04219467048911515

In [29]:
## trying multiple i values, but they're all worse
res = []
for i in range(1,10):
    sar = sm.tsa.statespace.SARIMAX(train_vix, exog=train_prime, order=(i,0,0), seasonal_order=(0,0,0,12), trend='c').fit()
    pred = sar.forecast(52*2,exog=validation_prime_df)
    res.append([i,RMSE(validation,pred)])
    

In [30]:
res

[[1, 0.04219467048911515],
 [2, 0.042198966438793166],
 [3, 0.042200886356342455],
 [4, 0.04219094036009502],
 [5, 0.042169889167493306],
 [6, 0.042139863468620545],
 [7, 0.04206785000526468],
 [8, 0.04207503231738511],
 [9, 0.04210447601011643]]

In [23]:
sar = sm.tsa.statespace.SARIMAX(train_vix, order=(1,0,0), seasonal_order=(0,0,0,12), trend='c').fit()
pred = sar.forecast(52*2)
RMSE(validation,pred)


0.04217228373626341